In [1]:
!unzip 'team13_bn.zip'

unzip:  cannot find or open team13_bn.zip, team13_bn.zip.zip or team13_bn.zip.ZIP.


In [2]:
import pandas as pd

data=pd.read_csv('task3/team13_bn_train.csv')

print(data['target'])

0        আজ শনিবার ভোররাতে ঢাকার হজরত শাহজালাল আন্তর্জা...
1                            কিন্তু আমরা সেটি করতে পারিনি।
2                                জীবন চলে তার নিজের গতিতে।
3                             চালককে পরে উদ্ধার করে পুলিশ।
4        ( ইব্রীয় ১০: ৩৬ - ৩৮) হাত শিথিল করার অথবা শয়তা...
                               ...                        
69995                   এই ব্যাপারে আমাদের আরো ভাবা জরুরি।
69996                                          বেইজ্জত কী?
69997    মন্ত্রী জানান, যাঁদের এখন চিকিৎসা চলছে, তাঁদের...
69998    এবং কেন নয়, যেহেতু তিনি এর সবেরই অভিজ্ঞতা লাভ ...
69999                             কংগ্রেসকে জবাব দিতে হবে।
Name: target, Length: 70000, dtype: object


In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import torch
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.translate.bleu_score import sentence_bleu
from transformers import AutoTokenizer, AutoModel

In [4]:
# Load GloVe embeddings
def load_glove_embeddings(glove_path):
  embeddings_index = {}
  with open(glove_path, 'r', encoding='utf-8') as f:
      for line in f:
          values = line.split()
          word = values[0]
          vector = np.asarray(values[1:], dtype='float32')
          embeddings_index[word] = vector
  return embeddings_index

# Prepare GloVe embedding matrix
def create_embedding_matrix(tokenizer, embedding_dim, embeddings_index):
  embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, embedding_dim))
  for word, i in tokenizer.word_index.items():
      embedding_vector = embeddings_index.get(word)
      if embedding_vector is not None:
          embedding_matrix[i] = embedding_vector
  return embedding_matrix

# Tokenize and pad sequences
def tokenize_and_pad(texts, tokenizer, max_len):
  sequences = tokenizer.texts_to_sequences(texts)
  padded = pad_sequences(sequences, maxlen=max_len, padding='post')
  return padded

def load_data(filename, with_references=False):
  data=pd.read_csv(filename)
  en=data.iloc[:, -2].values
  bn=data.iloc[:, -1].values

  return data,en,bn

def indicbert_embed(text, tokenizer, model, max_beng_len):
  input_ids = tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=max_beng_len).input_ids
  print(input_ids)
  outputs=[]
  with torch.no_grad():
    outputs = model(input_ids)
  print(outputs.last_hidden_state.numpy())
  return outputs.last_hidden_state.numpy()

In [5]:
# Load data
# Assume data is loaded into train_df, val_df, test_df with columns 'english' and 'bengali'
# Load training, validation and test data
train_data, train_en, train_bn = load_data("task3/team13_bn_train.csv")
val_data, val_en, val_bn = load_data("task3/team13_bn_valid.csv")
test_data, test_en, test_bn = load_data("task3/team13_bn_test.csv")

In [6]:
# Initialize English tokenizer
eng_tokenizer = Tokenizer()
eng_tokenizer.fit_on_texts(train_en)

# Load GloVe embeddings
glove_path = 'glove.6B/glove.6B.100d.txt'
embeddings_index = load_glove_embeddings(glove_path)
embedding_dim = 100
embedding_matrix = create_embedding_matrix(eng_tokenizer, embedding_dim, embeddings_index)

# Prepare English data
max_eng_len = max(len(seq.split()) for seq in train_en)
encoder_input_data = tokenize_and_pad(train_en, eng_tokenizer, max_eng_len)

In [7]:
# Initialize Bengali tokenizer using IndicBERT
bert_tokenizer = AutoTokenizer.from_pretrained('ai4bharat/indic-bert')
indicbert_model = AutoModel.from_pretrained('ai4bharat/indic-bert')

In [8]:
# Prepare Bengali data
max_beng_len = max(len(seq.split()) for seq in train_bn)
decoder_input_data = np.zeros((100, max_beng_len, indicbert_model.config.hidden_size))
print(decoder_input_data.shape)
for i, text in enumerate(train_bn):
  print(i)
  if i==100:
    break
  decoder_input_data[i] = indicbert_embed([text], bert_tokenizer, indicbert_model, max_beng_len)

(100, 201, 768)
0
tensor([[     2,   1089,      8,  10239,  30032,  23081,  32417,  33209,   8671,
          80619, 144742,  34480,    651,   5376,  26080,   2354, 116503,   4083,
          17498,  29716,  45428,  17223,    442,  29196,  30032,    442,   3865,
           6441,  33727,  46942,  10239,  41417,   2213,     15,      3,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
          

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


[[[-2.5745813e-02 -2.0128842e-02 -5.8500879e-03 ... -6.0939416e-03
   -2.2115324e-02  1.6428623e-04]
  [ 7.8883690e-01 -5.4826456e-01  2.1629146e-01 ... -1.1037045e-01
    1.2768424e-01 -1.6144864e-01]
  [ 8.2859278e-02 -3.4015286e-01 -3.8211495e-02 ...  1.9140303e-01
    3.5917699e-02  2.1419676e-01]
  ...
  [ 2.4122342e-01 -1.8304576e-01 -2.0505939e-01 ... -2.7510396e-01
   -1.9228370e-01 -5.7246846e-01]
  [ 2.5497645e-01 -1.9528218e-01 -2.0814160e-01 ... -2.6764077e-01
   -1.9024889e-01 -5.7103235e-01]
  [ 2.3461735e-01 -1.7985275e-01 -2.1161370e-01 ... -2.7365959e-01
   -1.8301447e-01 -5.6805509e-01]]]
1
tensor([[    2, 18788,   308,  8467,    78,  1847,  3681, 23974,   497,   382,
            15,     3,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,    

In [9]:
# Model architecture
latent_dim = 256

# Encoder
encoder_inputs = Input(shape=(None,), name='encoder_inputs')
encoder_embedding = Embedding(input_dim=len(eng_tokenizer.word_index) + 1, output_dim=embedding_dim,
                              weights=[embedding_matrix], input_length=max_eng_len, trainable=False)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True, name='encoder_lstm')
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

In [10]:
# Decoder
decoder_inputs = Input(shape=(None, indicbert_model.config.hidden_size), name='decoder_inputs')
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name='decoder_lstm')
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(indicbert_model.config.vocab_size, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_outputs)

In [11]:
# Define model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 encoder_inputs (InputLayer  [(None, None)]               0         []                            
 )                                                                                                
                                                                                                  
 embedding (Embedding)       (None, None, 100)            4535000   ['encoder_inputs[0][0]']      
                                                                                                  
 decoder_inputs (InputLayer  [(None, None, 768)]          0         []                            
 )                                                                                                
                                                                                              

In [12]:
# Prepare targets for training
decoder_target_data = np.zeros((100, max_beng_len, indicbert_model.config.vocab_size))
print(decoder_target_data.shape)
for i, text in enumerate(train_bn):
  if i==100:
    break
  for t, token_id in enumerate(bert_tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=max_beng_len).input_ids[0]):
    if t > 0:
      decoder_target_data[i, t - 1, token_id] = 1
print(decoder_target_data)

(100, 201, 200000)
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 

In [13]:
# Training
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=64, epochs=20)

ValueError: Data cardinality is ambiguous:
  x sizes: 70000, 100
  y sizes: 100
Make sure all arrays contain the same number of samples.

In [14]:
print(len(encoder_input_data))
print(len(decoder_input_data))
print(len(decoder_target_data))

70000
100
100


In [ ]:
# Inference models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_lstm_output, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_lstm_output)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [ ]:
# Translation function
def decode_sequence(input_seq):
  states_value = encoder_model.predict(input_seq)
  target_seq = np.zeros((1, 1, indicbert_model.config.hidden_size))
  stop_condition = False
  decoded_sentence = ''

  while not stop_condition:
      output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
      sampled_token_index = np.argmax(output_tokens[0, -1, :])
      sampled_word = bert_tokenizer.decode(sampled_token_index)
      decoded_sentence += ' ' + sampled_word

      if sampled_word == '[SEP]' or len(decoded_sentence) > max_beng_len:
          stop_condition = True

      target_seq = np.zeros((1, 1, indicbert_model.config.hidden_size))
      target_seq[0, 0, :] = output_tokens[0, -1, :]

      states_value = [h, c]

  return decoded_sentence

In [ ]:
# BLEU evaluation
from nltk.translate.bleu_score import corpus_bleu

def evaluate_bleu(model, data, eng_tokenizer, bert_tokenizer):
  references = []
  candidates = []

  for i, row in data.iterrows():
      input_seq = tokenize_and_pad([row['source']], eng_tokenizer, max_eng_len)
      decoded_sentence = decode_sequence(input_seq)
      references.append([row['source'].split()])
      candidates.append(decoded_sentence.split())

  bleu_scores = {
      'BLEU-1': corpus_bleu(references, candidates, weights=(1, 0, 0, 0)),
      'BLEU-2': corpus_bleu(references, candidates, weights=(0.5, 0.5, 0, 0)),
      'BLEU-3': corpus_bleu(references, candidates, weights=(0.33, 0.33, 0.33, 0)),
      'BLEU-4': corpus_bleu(references, candidates, weights=(0.25, 0.25, 0.25, 0.25))
  }

  return bleu_scores

# Evaluate on test data
test_bleu_scores = evaluate_bleu(model, test_data, eng_tokenizer, bert_tokenizer)
print(f"Test BLEU Scores: {test_bleu_scores}")
